# Exploration of Data.gov.sg's taxi and carpark availability API

Ref: https://beta.data.gov.sg/collections/85/view (carpark) ; https://beta.data.gov.sg/collections/352/view (taxi availability)

Data of interest: Realtime carpark and taxi availability.

## Notes:
- Carpark data only contains carpark ID without lat/lon location. A separate mapping file is required to facilitate location identification.
- For taxi availability, the coordinates and the total count are available.

In [50]:
from random import randint
from typing import Union, Dict
import requests
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

### Helper functions

In [58]:
def api_query(api_link: str,  agent_id: str) -> Union[Dict,None]:
    """Function which executes query via an api link using a provided agent_id as an identifier to avoid rejection of query request

    Args:
        api_link (str): API Link which requests is to be made
        agent_id (str): Id used for request header

    Returns:
        Dictioanry containing request content. None when exception are encountered.
    """
    req_headers = {"User-agent": agent_id}
    try:
        res = requests.get(api_link,
                            headers=req_headers,
                            timeout=5)
        # Raise if HTTPError occured
        res.raise_for_status()

        # Check the status code before extending the number of posts
        if res.status_code == 200:
            print(f"Request sucessful with {res.status_code}")
            the_json = res.json()
        return the_json

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)
    return None

def query_taxi_availability(api_link: str = "https://api.data.gov.sg/v1//transport/taxi-availability", agent_id: str= "test_qzq"): 
    taxi_avail_response = api_query(api_link=api_link, agent_id=agent_id)
    if not taxi_avail_response:
        print("No response received")
        return {}
    else:
        #print(taxi_avail_response)
        taxi_avail_response_features = taxi_avail_response["features"][0]
        geo_coord = taxi_avail_response_features["geometry"]["coordinates"]
        geo_coord_lat_lon = [lat_lon[::-1] for lat_lon in geo_coord]
    return geo_coord_lat_lon



def query_carpark_availability(api_link: str = "https://api.data.gov.sg/v1/transport/carpark-availability", agent_id: str= "test_qzq"): 
    carpark_avail_response = api_query(api_link=api_link, agent_id=agent_id+str(randint(0,100)))

    carpark_dict = {}
    if not carpark_avail_response:
        print("No response received")
        
    else:
        carpark_data_list = carpark_avail_response["items"][0]["carpark_data"]
        # There maybe possible duplicated carpark number in request, hence overall carpark info may be lesser then the response list length
        for carpark_info_dict in carpark_data_list:
            carpark_capacity_info = carpark_info_dict["carpark_info"][0]
            carpark_number = carpark_info_dict.pop("carpark_number")
            carpark_dict[carpark_number] = carpark_capacity_info
    return carpark_dict

### Function call to explore response outputs

#### Taxi availability

In [61]:
# Query taxi availability
taxi_availability = query_taxi_availability()
len(taxi_availability)

Request sucessful with 200


2593

In [62]:
# Filter within 500m of test point
from geopy.distance import geodesic

radius = 0.5 #in km
test_point = tuple([1.3,103.86])

# Filter for coordinates which are within a specified radius of test point
surrounding_radius_taxi = [tuple(taxi_location) for taxi_location in taxi_availability if geodesic(test_point,tuple(taxi_location)).kilometers
< radius]
len(surrounding_radius_taxi)

26

#### Carpark availability

In [33]:
# Possible time out response when multiple attempts of querying are made.
carpark_availability = query_carpark_availability()
print(len(carpark_availability))
carpark_availability 

Request sucessful with 200
1948
[{'carpark_info': [{'total_lots': '105', 'lot_type': 'C', 'lots_available': '0'}], 'carpark_number': 'HE12', 'update_datetime': '2023-11-17T09:23:47'}, {'carpark_info': [{'total_lots': '583', 'lot_type': 'C', 'lots_available': '263'}], 'carpark_number': 'HLM', 'update_datetime': '2023-11-17T09:23:51'}, {'carpark_info': [{'total_lots': '329', 'lot_type': 'C', 'lots_available': '149'}], 'carpark_number': 'RHM', 'update_datetime': '2023-11-17T09:23:47'}, {'carpark_info': [{'total_lots': '97', 'lot_type': 'C', 'lots_available': '11'}], 'carpark_number': 'BM29', 'update_datetime': '2023-11-17T09:23:41'}, {'carpark_info': [{'total_lots': '96', 'lot_type': 'C', 'lots_available': '0'}], 'carpark_number': 'Q81', 'update_datetime': '2023-11-17T09:23:50'}, {'carpark_info': [{'total_lots': '176', 'lot_type': 'C', 'lots_available': '31'}], 'carpark_number': 'C20', 'update_datetime': '2023-11-17T09:24:02'}, {'carpark_info': [{'total_lots': '228', 'lot_type': 'C', 'lot

{'HE12': {'total_lots': '105', 'lot_type': 'C', 'lots_available': '0'},
 'HLM': {'total_lots': '583', 'lot_type': 'C', 'lots_available': '263'},
 'RHM': {'total_lots': '329', 'lot_type': 'C', 'lots_available': '149'},
 'BM29': {'total_lots': '97', 'lot_type': 'C', 'lots_available': '11'},
 'Q81': {'total_lots': '96', 'lot_type': 'C', 'lots_available': '0'},
 'C20': {'total_lots': '176', 'lot_type': 'C', 'lots_available': '31'},
 'FR3M': {'total_lots': '228', 'lot_type': 'C', 'lots_available': '4'},
 'C32': {'total_lots': '289', 'lot_type': 'C', 'lots_available': '238'},
 'C6': {'total_lots': '332', 'lot_type': 'C', 'lots_available': '183'},
 'TG2': {'total_lots': '273', 'lot_type': 'C', 'lots_available': '145'},
 'BP1': {'total_lots': '577', 'lot_type': 'C', 'lots_available': '365'},
 'TG1': {'total_lots': '133', 'lot_type': 'C', 'lots_available': '97'},
 'TGM2': {'total_lots': '189', 'lot_type': 'C', 'lots_available': '139'},
 'TE14': {'total_lots': '134', 'lot_type': 'C', 'lots_avail

# Enhance retreived carpark availability with Lat Lon information derived from original HDB Carpark information

In [44]:
hdb_carpark_info = pd.read_csv("../data/HDBCarparkInformationWGS84.csv", index_col="car_park_no")

default_lot_info_dict = {'total_lots': 'No info', 'lot_type': 'No info', 'lots_available': 'No info'}

# Set default values 
hdb_carpark_info = hdb_carpark_info.assign(**default_lot_info_dict)
hdb_carpark_info.head()


,address,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement,Lat,Lon,total_lots,lot_type,lots_available
car_park_no,,,,,,,,,,,,,,
ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.8,Y,1.290431,103.864682,No info,No info,No info
ACM,BLK 98A ALJUNIED CRESCENT,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.1,N,1.321573,103.884496,No info,No info,No info
AH1,BLK 101 JALAN DUSUN,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.0,N,1.280870,103.891727,No info,No info,No info
AK19,BLOCK 253 ANG MO KIO STREET 21,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.0,N,1.271172,103.932271,No info,No info,No info
AK31,BLK 302/348 ANG MO KIO STREET 31,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N,1.282898,103.929320,No info,No info,No info


In [63]:
default_lot_info_dict = {'total_lots': 'No info', 'lot_type': 'No info', 'lots_available': 'No info'}

# Set default 

# Set default values 
hdb_carpark_info = hdb_carpark_info.assign(**default_lot_info_dict)
hdb_carpark_info.head()

hdb_carpark_dict = hdb_carpark_info.to_dict('index')
# Update carpark availability info with lat lon info
hdb_carpark_dict.update(carpark_availability)
len(hdb_carpark_dict)

2221